### Poisson Ratings
https://penaltyblog.readthedocs.io/en/latest/ratings/massey_ratings.html

https://www.pro-football-reference.com/years/2022/games.htm


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
import os
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import penaltyblog as pb

In [5]:
try:
    import nfl_data_py as nfl
except:
    ! pip install nfl_data_py

In [4]:
import nfl_data_py as nfl

In [6]:
semana = 7
temporada = 2022
normalizacion = 1
MAX = 10
df = nfl.import_schedules([temporada])
df = df[df['week']<semana]

In [7]:
df = df.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})


In [8]:
if normalizacion == 1:
    df['FTHG'] = (df['FTHG'] - df['FTHG'].min()) / 
                ( df['FTHG'].max() - df['FTHG'].min()) * MAX
    df['FTAG'] = (df['FTAG'] - df['FTAG'].min()) / ( 
                 df['FTAG'].max() - df['FTAG'].min()) * MAX

In [9]:
df[["FTHG", "FTAG"]].mean()

FTHG    4.392730
FTAG    4.566869
dtype: float64

In [10]:
pr = nfl.import_schedules([temporada])
pr = pr[pr['week']==semana ]
juegos = pr['week'].count()

In [11]:
pr = pr.rename(columns={'away_team':'AwayTeam',
                                   'home_team':'HomeTeam',
                       'away_score':'FTHG',
                       'home_score':'FTAG'})

In [12]:
pois = pb.models.PoissonGoalsModel(df["FTHG"], df["FTAG"], 
                         df["HomeTeam"], df["AwayTeam"])
pois.fit()

In [16]:
print('Poisson Model')
print(f'Season: {temporada}')
print(f'Week: {semana}')
for i in range(0,juegos):
    EL = pr.iloc[i]['HomeTeam']
    EV = pr.iloc[i]['AwayTeam']
    probs = pois.predict(EL, EV)
    #print(f'Home {EL} {probs.home_win} vs Away {EV} {probs.away_win}')
    if probs.home_win >= .50 :
        print(f'L {EL} -{100/probs.home_win:.0f} vs V {EV} {100/probs.away_win:.0f}')
    elif probs.away_win >= .50:
        print(f'L {EL} {100/probs.home_win:.0f} vs V {EV} -{100/probs.away_win:.0f}')
    else:
        print(f'L {EL} {100/probs.home_win:.0f} vs V {EV} {100/probs.away_win:.0f}')

Poisson Model
Season: 2022
Week: 7
L ARI -197 vs V NO 280
L BAL 1204 vs V CLE -122
L CAR -143 vs V TB 590
L CIN 406 vs V ATL -161
L DAL 429 vs V DET -151
L JAX 237 vs V NYG 233
L TEN 243 vs V IND 226
L WAS 208 vs V GB 279
L DEN -161 vs V NYJ 455
L LV 258 vs V HOU 209
L LAC 285 vs V SEA -191
L SF 276 vs V KC -199
L MIA 579 vs V PIT -137
L NE 832 vs V CHI -130


In [14]:
pois

Module: Penaltyblog

Model: Poisson

Number of parameters: 65
Log Likelihood: -358.726
AIC: 847.452

Team                 Attack               Defence             
------------------------------------------------------------
ARI                  1.045                0.08                
ATL                  1.079                0.412               
BAL                  0.998                0.868               
BUF                  0.269                0.62                
CAR                  1.182                0.082               
CHI                  1.131                -0.009              
CIN                  0.723                0.421               
CLE                  1.338                0.517               
DAL                  0.73                 0.349               
DEN                  0.75                 -0.175              
DET                  1.555                0.831               
GB                   1.094                0.257               
HOU                

In [15]:
pois.get_params()

{'attack_ARI': 1.0453658178094067,
 'attack_ATL': 1.079431187363565,
 'attack_BAL': 0.9984436296079033,
 'attack_BUF': 0.2688094985273027,
 'attack_CAR': 1.182260923713708,
 'attack_CHI': 1.1307872236457814,
 'attack_CIN': 0.7232213994743643,
 'attack_CLE': 1.3377832596220653,
 'attack_DAL': 0.7303150334380958,
 'attack_DEN': 0.7503296652662854,
 'attack_DET': 1.5551114986431522,
 'attack_GB': 1.0935078640261373,
 'attack_HOU': 1.1074404679362109,
 'attack_IND': 0.9478952681616304,
 'attack_JAX': 1.0007955078737478,
 'attack_KC': 1.2948960328829227,
 'attack_LA': 1.0702087622867784,
 'attack_LAC': 1.1292565821928817,
 'attack_LV': 1.2931508919724986,
 'attack_MIA': 1.1068206238799738,
 'attack_MIN': 0.830568773249577,
 'attack_NE': 0.6055834755569847,
 'attack_NO': 1.308089331726023,
 'attack_NYG': 0.8829374289184715,
 'attack_NYJ': 0.8614924824609496,
 'attack_PHI': 0.7857559692981849,
 'attack_PIT': 1.160982514739654,
 'attack_SEA': 1.3130995999825086,
 'attack_SF': 0.594007532382930